In [1]:
using Base.Threads

In [2]:
Threads.nthreads()

64

In [3]:
pwd()

"/home/xyu/github/BulkLMM.jl/analysis/Explore_Multithreading"

In [14]:
cd("..")

In [15]:
include("../test/BXDdata_for_test.jl");

In [16]:
include("../src/parallel_helpers.jl");

In [17]:
include("../test/testHelper.jl");

In [18]:
# using Pkg
# Pkg.add("ThreadTools")
using ThreadTools

In [19]:
?tmap

search: tmap tmap1 @atomicswap BitMatrix Cintmax_t floatmax @fastmath Cuintmax_t



```
tmap(f, args...)
tmap(f, v, T::DataType)
tmap(f, nthreads::Int, args...)
tmap1(f, args...)
tmap1(f, nthreads::Int, args...)
```

Threaded map. The optional argument `nthreads` limits the number of threads used in parallel. `tmap1` is the same as `tmap`, but falls back to a regular `map` if julia only has access to one thread. If the eltype `T` of the output is specified the call will be type stable.


In [55]:
function threads12_by_blocks(r0perm::Array{Float64, 2}, X00::Array{Float64, 2}, nblocks::Int64)
    # Does distributed processes of calculations of LOD scores for markers in each block

    p = size(X00, 2);

    ## (Create blocks...)
    
    block_size = ceil(Int, p/nblocks);
    blocks = createBlocks2(p, block_size);
    # blocks = createBlocks(p, nblocks);

    LODs_blocks = tmap(x -> calcLODs_block(r0perm, X00, x), 64, blocks);
    results = reduce(hcat, LODs_blocks);

    return results

end

threads12_by_blocks (generic function with 1 method)

In [56]:
function scan_perms_threads12(y::Array{Float64,2}, g::Array{Float64,2}, K::Array{Float64,2};
                                reml::Bool = false,
                                nperms::Int64 = 1024, rndseed::Int64 = 0, original::Bool = true,
                                # (options for blocks, nperms distribution methods...)
                                option::String = "by blocks", nblocks::Int64 = 1, ncopies::Int64 = 1, 
                                nprocs::Int64 = 0)

    # addprocs(nprocs);
    # @everywhere using ParallelDataTransfer
    # sendto(workers(), y = y);
    #sendto(workers(), g = g);
    #sendto(workers(), K = K);
    #sendto(workers(), reml = reml);

    #@everywhere begin
        #include("../src/parallel_helpers.jl"); # for now; needs to be revised later
        #(y0, X0, lambda0) = transform1(y, g, K); # rotation of data
        # (r0, X00) = transform2(y0, X0, lambda0; reml = reml); # reweighting and taking residuals
    
    #end

    (y0, X0, lambda0) = transform_rotation(y, g, K); # rotation of data
    (r0, X00) = transform_reweight(y0, X0, lambda0; reml = reml); # reweighting and taking residuals
    r0perm = transform_permute(r0; nperms = nperms, rndseed = rndseed, original = original);

    if option == "by blocks"
        results = threads12_by_blocks(r0perm, X00, nblocks);
    elseif option == "by nperms"
        results = distribute_by_nperms(r0, X00, nperms, ncopies, original);
    else
        throw(error("Option unsupported."))
    end

    return results

end

scan_perms_threads12 (generic function with 1 method)

In [58]:
@benchmark scan_perms_threads12(pheno_y, geno, kinship; reml = false, nperms = 1024, rndseed = 0, original = true,
                                             option = "by blocks", nblocks = 320)

BenchmarkTools.Trial: 4 samples with 1 evaluation.
 Range (min … max):  1.166 s …    1.629 s  ┊ GC (min … max):  9.99% … 22.96%
 Time  (median):     1.520 s               ┊ GC (median):    11.65%
 Time  (mean ± σ):   1.459 s ± 203.212 ms  ┊ GC (mean ± σ):  14.48% ±  6.02%

  █                                       █      █         █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁█ ▁
  1.17 s         Histogram: frequency by time         1.63 s <

 Memory estimate: 13.68 GiB, allocs estimate: 81091.

In [91]:
@benchmark scan_perms_threads12(pheno_y, geno, kinship; reml = false, nperms = 1024, rndseed = 0, original = true,
                                             option = "by blocks", nblocks = 160)

BenchmarkTools.Trial: 4 samples with 1 evaluation.
 Range (min … max):  1.183 s …    1.846 s  ┊ GC (min … max):  2.44% … 33.00%
 Time  (median):     1.542 s               ┊ GC (median):    13.40%
 Time  (mean ± σ):   1.528 s ± 357.844 ms  ┊ GC (mean ± σ):  17.20% ± 12.80%

  █     █                                                 ██  
  █▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁██ ▁
  1.18 s         Histogram: frequency by time         1.85 s <

 Memory estimate: 13.67 GiB, allocs estimate: 77918.

In [92]:
@benchmark scan_perms(pheno_y, geno, kinship; reml = false, nperms = 1024, rndseed = 0, original = true)

BenchmarkTools.Trial: 2 samples with 1 evaluation.
 Range (min … max):  2.845 s …   2.887 s  ┊ GC (min … max): 6.51% … 6.24%
 Time  (median):     2.866 s              ┊ GC (median):    6.38%
 Time  (mean ± σ):   2.866 s ± 30.209 ms  ┊ GC (mean ± σ):  6.38% ± 0.19%

  █                                                       █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  2.84 s         Histogram: frequency by time        2.89 s <

 Memory estimate: 13.61 GiB, allocs estimate: 74659.

In [59]:
tmap_LODs = scan_perms_threads12(pheno_y, geno, kinship; reml = false, nperms = 1024, rndseed = 0, original = true,
                                             option = "by blocks", nblocks = 320);

In [60]:
using Distributed

In [64]:
# addprocs(4);

In [65]:
nprocs()

17

In [66]:
@everywhere begin
    using Random
    using LinearAlgebra
    using Optim
    using Distributed
    
    ## Include the source code of BulkLMM to be tested:
    include("../src/scan.jl");
    include("../src/lmm.jl");
    include("../src/wls.jl");
    include("../src/util.jl");
    include("../src/kinship.jl");
    include("../src/readData.jl");
    include("../src/scan_for_tests.jl")
    include("../src/scan_distributed.jl")

    include("../src/parallel_helpers.jl")
end

In [90]:
pmap_LODs = scan_perms_distributed(pheno_y, geno, kinship; reml = false, nperms = 1024, rndseed = 0, original = true,
                                             option = "by blocks", nblocks = 160);

In [75]:
@time scan_perms_distributed(pheno_y, geno, kinship; reml = false, nperms = 1024, rndseed = 0, original = true,
                                             option = "by blocks", nblocks = 120);

  0.776413 seconds (12.36 k allocations: 143.207 MiB, 8.94% gc time)


In [87]:
@benchmark scan_perms_distributed(pheno_y, geno, kinship; reml = false, nperms = 1024, rndseed = 0, original = true,
                                             option = "by blocks", nblocks = 160)

BenchmarkTools.Trial: 8 samples with 1 evaluation.
 Range (min … max):  633.504 ms … 699.389 ms  ┊ GC (min … max): 0.89% … 0.00%
 Time  (median):     647.621 ms               ┊ GC (median):    0.32%
 Time  (mean ± σ):   656.485 ms ±  25.549 ms  ┊ GC (mean ± σ):  0.39% ± 0.43%

  █ ▁▁                  ▁         ▁              ▁            ▁  
  █▁██▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  634 ms           Histogram: frequency by time          699 ms <

 Memory estimate: 143.34 MiB, allocs estimate: 15800.

In [88]:
pmap_LODs

1025×7321 Matrix{Float64}:
 0.00819636  0.00819636  0.00819636  …  0.0449878    0.0128283   0.0128283
 0.604956    0.604956    0.604956       1.33928      1.29426     1.29426
 0.0264842   0.0264842   0.0264842      0.16927      0.069518    0.069518
 0.00585467  0.00585467  0.00585467     0.589608     0.14854     0.14854
 0.477297    0.477297    0.477297       0.0317133    0.0625366   0.0625366
 0.140293    0.140293    0.140293    …  0.0400303    0.0338204   0.0338204
 0.45141     0.45141     0.45141        0.459152     0.278652    0.278652
 0.179818    0.179818    0.179818       0.0430329    0.0218039   0.0218039
 0.213351    0.213351    0.213351       0.179031     0.193549    0.193549
 0.0250041   0.0250041   0.0250041      0.0679043    0.00622826  0.00622826
 0.801668    0.801668    0.801668    …  0.42936      0.123412    0.123412
 0.0351507   0.0351507   0.0351507      0.296432     0.0480549   0.0480549
 0.053255    0.053255    0.053255       0.281489     0.11254     0.11254
 ⋮     

In [89]:
# Check the results from tmap and pmap:
sumSqDiff(tmap_LODs, pmap_LODs)

2.717660331150989e-26